<a href="https://colab.research.google.com/github/nakulkd/COVID-19-TweetIDs/blob/master/CDA_Project_Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Initialization

In [3]:
GIT_USERNAME = "nakulkd"
GIT_TOKEN = ""
GIT_REPOSITORY = "COVID-19-TweetIDs"
GIT_PATH = f"https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"

In [6]:
!git clone "{GIT_PATH}"

Cloning into 'COVID-19-TweetIDs'...
remote: Enumerating objects: 7312, done.
remote: Total 7312 (delta 0), reused 0 (delta 0), pack-reused 7312
Receiving objects: 100% (7312/7312), 4.58 GiB | 17.67 MiB/s, done.
Resolving deltas: 100% (238/238), done.
Checking out files: 100% (6929/6929), done.


In [8]:
!dir

COVID-19-TweetIDs  sample_data


# Directory Walk

In [9]:
import os

In [10]:
os.chdir("COVID-19-TweetIDs")

In [27]:
os.chdir("./2020-01")

In [28]:
fileList = os.listdir()

In [31]:
folder_name = '2020-01'

In [55]:
!rm test_sampler.csv

In [58]:
with open('test_sampler.txt', 'w') as cursor:
    i = 0
    while i < 10:
        with open(fileList[i], 'r') as tweet:
            tweet_count = 0
            for line in tweet:
                cursor.write(f"{folder_name},{fileList[i]},{line}")
                tweet_count += 1
                if tweet_count == 10:
                    break
        i += 1

In [59]:
counter = 0
with open('test_sampler.txt', 'r') as point:
    for line in point:
        counter += 1

In [60]:
counter

100

## Tweet ID Sampler

In [87]:
folder_list = ['2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11']

In [108]:
## Sampler

file_count = 20
tweets = 250

with open('sampled_tweets.txt', 'w') as cursor: # Create sample holder file as CSV
    for folder in folder_list: # Loop through each folder
        print(f"Currently in Folder '{folder}'")
        os.chdir(f'./{folder}') # Enter Folder
        fileList = os.listdir() # List the files
        i = 0 # Init the file loop index
        while i < file_count: # For 10 files in each folder...
            with open(fileList[i], 'r') as tweet:
                tweet_count = 0
                for line in tweet:
                    cursor.write(f"{folder},{fileList[i]},{line}") # ... Write Tweet ID and corresponding metadata...
                    tweet_count += 1
                    if tweet_count == tweets: # ... Upto 10 entries per file
                        break
            i += 1
        os.chdir('..') # Exit Folder

Currently in Folder '2020-02'
Currently in Folder '2020-03'
Currently in Folder '2020-04'
Currently in Folder '2020-05'
Currently in Folder '2020-06'
Currently in Folder '2020-07'
Currently in Folder '2020-08'
Currently in Folder '2020-09'
Currently in Folder '2020-10'
Currently in Folder '2020-11'


In [109]:
counter = 0
with open('sampled_tweets.txt', 'r') as point:
    for line in point:
        counter += 1

print(counter)

50000


# Rehydrating Tweets

In [91]:
!pip3 install twarc
!pip3 install tqdm

  Created wheel for twarc: filename=twarc-1.12.0-cp36-none-any.whl size=25297 sha256=6444ed1fd5817c8b95fa74902cf5a3a89c4506f6bd0c6dafd5fa20d1988bae3c
  Stored in directory: /root/.cache/pip/wheels/eb/c9/a3/a5d1f92bb65159f864294a65d30adea23d453144a31754cda9
Successfully built twarc


In [ ]:
!twarc configure

In [95]:
import gzip
import json

from tqdm import tqdm
from twarc import Twarc
from pathlib import Path

twarc = Twarc()

In [96]:
def _reader_generator(reader):
    b = reader(1024 * 1024)
    while b:
        yield b
        b = reader(1024 * 1024)


def raw_newline_count(fname):
    """
    Counts number of lines in file
    """
    f = open(fname, 'rb')
    f_gen = _reader_generator(f.raw.read)
    return sum(buf.count(b'\n') for buf in f_gen)


def hydrate(id_file):
    print('hydrating {}'.format(id_file))

    gzip_path = id_file.with_suffix('.jsonl.gz')
    if gzip_path.is_file():
        print('skipping json file already exists: {}'.format(gzip_path))
        return

    num_ids = raw_newline_count(id_file)

    with gzip.open(gzip_path, 'w') as output:
        with tqdm(total=num_ids) as pbar:
            for tweet in twarc.hydrate(id_file.open()):
                output.write(json.dumps(tweet).encode('utf8') + b"\n")
                pbar.update(1)

In [ ]:
data_dirs = ['Samples']
for data_dir in data_dirs:
    for path in Path(data_dir).iterdir():
        if path.name.endswith('.txt'):
            hydrate(path)

In [116]:
os.chdir('./Samples')

In [ ]:
!twarc hydrate sampled_tweets_only.txt > tweets.jsonl

In [ ]:
json_content = []
filename = 'sampled_tweets.jsonl.gz'

with gzip.open(filename , 'rb') as gzip_file:
    for line in gzip_file:  # Read one line.
        line = line.rstrip()
        if line:  # Any JSON data on it?
            obj = json.loads(line)
            json_content.append(obj)

In [105]:
len(json_content)

8457

In [113]:
os.chdir('..')
os.getcwd()

'/content/COVID-19-TweetIDs'